In [7]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image

In [8]:
#parameter
sizeImage = 784
h_dim = 400
z_dim = 20
epochsNum = 50 #number of epochs
batch_size = 100
lr = 0.01

sampleLocation = 'samplesOfVAE'
if not os.path.exists(sampleLocation):
    os.makedirs(sampleLocation)
    
torch.cuda.set_device(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
#download MNIST
dataset =torchvision.datasets.MNIST(root='dataMNIST', train=True, transform=transforms.ToTensor(), download=True)#If already downloaded, download=False
data_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size,shuffle=True)

In [10]:
class VAE(nn.Module):
    def __init__(self,sizeImage=784, h_dim=400, z_dim=20):
        super(VAE, self).__init__()
        self.fc1=nn.Linear(sizeImage, h_dim)
        self.fc2=nn.Linear(h_dim, z_dim)
        self.fc3=nn.Linear(h_dim, z_dim)
        self.fc4=nn.Linear(z_dim, h_dim)
        self.fc5=nn.Linear(h_dim, sizeImage)
    
    def encode(self, x):
        h=F.relu(self.fc1(x))
        return self.fc2(h),self.fc3(h)

    def decode(self, z):
        h=F.relu(self.fc4(z))
        return F.sigmoid(self.fc5(h))
    
    def forward(self, x):
        mu, log_var =self.encode(x)
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        z = mu +eps *std
        x_reconst =self.decode(z)
        return x_reconst, mu, log_var

In [11]:
model = VAE().to(device)
optimizer =torch.optim.Adam(model.parameters(), lr=lr)

In [12]:
for epoch in range(epochsNum):
    model.train()
    for i, (x, _) in enumerate(data_loader):
        # forward
        model.zero_grad()
        x = x.to(device).view(-1, sizeImage)
        x_reconst, mu, log_var = model(x)
        
        # Compute reconstruction loss and kl divergence
        reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
        kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        
        # backward
        loss = reconst_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch[{}/{}], Step [{}/{}], reconst_loss: {:.4f}, kl_div: {:.4f}" 
                   .format(epoch+1, epochsNum, i+1, len(data_loader), reconst_loss.item(), kl_div.item()))
    
    with torch.no_grad():
        # save sample image
        z = torch.randn(batch_size, z_dim).to(device)
        out = model.decode(z).view(-1, 1, 28, 28)
        save_image(out, os.path.join(sampleLocation, 'sampled-{}.png'.format(epoch+1)))

        # save reconstructed image
        out, _, _ = model(x)
        x_concat = torch.cat([x.view(-1, 1, 28, 28), out.view(-1, 1, 28, 28)], dim=3)
        save_image(x_concat, os.path.join(sampleLocation, 'reconst-{}.png'.format(epoch+1)))

D:\Software\Anaconda3\lib\site-packages\torch\nn\functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
D:\Software\Anaconda3\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch[1/2], Step [100/600], reconst_loss: 15381.2070, kl_div: 1276.9407
Epoch[1/2], Step [200/600], reconst_loss: 12712.6113, kl_div: 1462.7153
Epoch[1/2], Step [300/600], reconst_loss: 12302.5059, kl_div: 1429.1528
Epoch[1/2], Step [400/600], reconst_loss: 12068.9023, kl_div: 1577.8577
Epoch[1/2], Step [500/600], reconst_loss: 11372.7441, kl_div: 1701.8625
Epoch[1/2], Step [600/600], reconst_loss: 10640.4414, kl_div: 1641.7629
Epoch[2/2], Step [100/600], reconst_loss: 10502.5654, kl_div: 1719.1991
Epoch[2/2], Step [200/600], reconst_loss: 10154.3447, kl_div: 1705.4841
Epoch[2/2], Step [300/600], reconst_loss: 10430.1143, kl_div: 1712.0306
Epoch[2/2], Step [400/600], reconst_loss: 10659.3516, kl_div: 1779.5876
Epoch[2/2], Step [500/600], reconst_loss: 10493.8721, kl_div: 1766.5500
Epoch[2/2], Step [600/600], reconst_loss: 10262.7695, kl_div: 1783.9532
